In [33]:
# Imports and global tolerance
import mercury as mr
import random
import csv

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import numpy as np

tolerance = 1e-7

In [34]:
# User interface inputs
show_code = mr.Checkbox(value=False, label="Show source code")
app = mr.App(
    title="Loan House Calculation",
    description="Actuarial Mathematics",
    show_code=show_code.value,
    continuous_update=True
)

admin_toggle = mr.Checkbox(value=False, label="Enable Admin Mode (Simulate Random Contracts)")
details_toggle = mr.Checkbox(value=False, label="Show detailed calculations")
_ = mr.Note(text="---")

if admin_toggle.value == True:
    num_contracts = mr.Numeric(label="Number of Random Contracts", value=10, min=1, max=1000)
else:
    age = mr.Numeric(label="Age (Years)", value=22, min=18, max=100)
    amount = mr.Numeric(label="Loan Amount", min=0, value=150000, max=999999999999)
    interest_rate = mr.Numeric(label="Yearly Interest Rate (%)", value=2.5, min=0.0, max=100.0, step=0.1)
    maturity = mr.Numeric(label="Maturity (Years)", value=25, min=1, max=50)

mortality_table = mr.Select(value = "TD88-90 (France)", choices = ["TD88-90 (France)", "TH00-02 (France)"], label = "Select Mortality Table")
currency = mr.Select(value = "EUR", choices = ["EUR", "USD", "GBP"], label = "Select Currency")

mercury.Checkbox

mercury.App

mercury.Checkbox

---

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Select

mercury.Select

In [35]:
_ = mr.Note(text="---")
_ = mr.Note("Please fill out the inputs above and click _Apply_ or hit Enter to calculate the result.")

---

Please fill out the inputs above and click _Apply_ or hit Enter to calculate the result.

In [36]:
# Load mortality table
def load_mortality_data(filename):
    with open(filename, "r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        return {int(row[0]): int(row[1]) for row in reader}

if mortality_table.value == "TD88-90 (France)":
    mortality_data = load_mortality_data("TD88-90.csv")
elif mortality_table.value == "TH00-02 (France)":
    mortality_data = load_mortality_data("TH00-02.csv")

In [37]:
# Calculation helper functions
def annuity_factor(x, m, tech_rate):
    act_sum = 0
    for i in range(0, m):
        act_sum += nex(x, i, tech_rate)
    return act_sum

def nex(x, n, tech_rate):
    return npx(x, n) * techDF(n, tech_rate)

def npx(x ,n):
    return mortality_data[x+n] / mortality_data[x]

def techDF(n, tech_rate):
    return 1 / ((1 + tech_rate) ** n)

def n_1qx(x, n):
    return (mortality_data[x+n] - mortality_data[x+n+1]) / mortality_data[x]

In [38]:
# Core logic
def calculate_contract(p_age, p_maturity, p_interest, p_amount):
    number_of_payments = int(p_maturity * 12) # input is in years
    periodic_rate = p_interest / 12 / 100 # input is yearly + should be divided by 100
    monthly_reimbursed_amount = (p_amount * periodic_rate) / (1 - (1 + periodic_rate) ** -number_of_payments)

    remaining_amount = []
    interest = []
    amortization = []

    for i in range(0, number_of_payments + 1):
        # Special cases on the 0 month
        if i == 0:
            remaining_amount.append(p_amount)
            interest.append(0)
            amortization.append(0)
            continue

        # Remaining amount (end of the year) Ck
        # Using tolerance for the last element (not perfect zero)
        calculated_value = (1 + periodic_rate) * remaining_amount[-1] - monthly_reimbursed_amount
        remaining_amount.append(0 if calculated_value < tolerance else calculated_value)

        # Interest Ik
        interest.append(remaining_amount[i-1] * periodic_rate)
        # Amortization Ak
        amortization.append(monthly_reimbursed_amount - interest[i])

    total_reimbursement_amount = sum(amortization)
    cost_of_loan = sum(interest)
    # print(f"Total reimbursement amount: {total_reimbursement_amount:.2f} {currency.value}")
    # print(f"Cost of loan: {cost_of_loan:.2f} {currency.value}")

    # Yearly payment calculations
    k = 0
    discounted_amount_in_risks = []

    for i in range(1, len(remaining_amount)):
        if i % 12 == 0:
            benefit_akprime = remaining_amount[i]
            discount_factor = (1 / (1 + p_interest / 100)) ** (k+1)
            probability = (mortality_data[p_age+k] - mortality_data[p_age+k+1]) / mortality_data[p_age]
            discounted_amount_in_risk = benefit_akprime * discount_factor * probability
            discounted_amount_in_risks.append(discounted_amount_in_risk)
            k += 1
    total_discounted_amount_in_risk = sum(discounted_amount_in_risks)
    single_premium = total_discounted_amount_in_risk

    annuity_factor_calc = annuity_factor(int(p_age), int(p_maturity), p_interest / 100)
    annual_premium = single_premium / annuity_factor_calc

    # Monthly payment calculations
    monthly_annuity_factor = annuity_factor_calc - ((12 - 1 )/( 2 * 12)) * (1 - nex(p_age, p_maturity, p_interest / 100))
    monthly_annual_premium = single_premium / monthly_annuity_factor
    monthly_premium = monthly_annual_premium / 12

    return single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, number_of_payments, periodic_rate, monthly_reimbursed_amount, remaining_amount, interest, amortization, total_reimbursement_amount, cost_of_loan


In [51]:
# Simulation helper functions
def evaluate_model(y_test, y_pred, target_name):
    print(f"{target_name} Prediction Performance:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
    print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")
    print(f"R² Score: {r2_score(y_test, y_pred):.4f}\n")

def plot_learning_curve():
    sizes = list(range(10, len(X), 10))
    r2_scores_y1 = []
    r2_scores_y2 = []

    for size in sizes:
        X_sample, y1_sample, y2_sample = X_scaled[:size], y1[:size], y2[:size]

        if size < 20:  # Avoid unreliable training with too small datasets
            r2_scores_y1.append(np.nan)
            r2_scores_y2.append(np.nan)
            continue

        model1_cv = RandomForestRegressor(n_estimators=100, random_state=42)
        model2_cv = RandomForestRegressor(n_estimators=100, random_state=42)

        r2_y1 = np.mean(cross_val_score(model1_cv, X_sample, y1_sample, cv=5, scoring='r2'))
        r2_y2 = np.mean(cross_val_score(model2_cv, X_sample, y2_sample, cv=5, scoring='r2'))

        r2_scores_y1.append(r2_y1)
        r2_scores_y2.append(r2_y2)

    plt.figure(figsize=(10, 5))
    plt.plot(sizes, r2_scores_y1, marker='o', label='Single Premium R² Score')
    plt.plot(sizes, r2_scores_y2, marker='o', label='Annual Premium R² Score')

    # Mark points above threshold
    for size, r2_y1, r2_y2 in zip(sizes, r2_scores_y1, r2_scores_y2):
        if r2_y1 >= 0.90:
            plt.scatter(size, r2_y1, color='green', s=100, edgecolors='black', label='_nolegend_')
        if r2_y2 >= 0.90:
            plt.scatter(size, r2_y2, color='green', s=100, edgecolors='black', label='_nolegend_')

    plt.axhline(y=0.90, color='r', linestyle='--', label='90% Threshold')
    plt.xlabel("Dataset Size")
    plt.ylabel("R² Score")
    plt.title("Learning Curve - R² Score vs Dataset Size (Cross-Validated)")
    plt.legend()
    plt.grid()
    plt.ylim(bottom=0)
    plt.show()

In [40]:
# If admin simulation
if admin_toggle.value == True:
    random_contracts = []
    for _ in range(int(num_contracts.value)):
        age = random.randint(18, 90)
        loan_amount = random.randint(10000, 1000000)
        interest_rate = random.uniform(0.5, 15.0)
        maturity = random.randint(1, 100 - age) # upper do not to overflow the mortality table

        single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, number_of_payments, periodic_rate, monthly_reimbursed_amount, remaining_amount, interest, amortization, total_reimbursement_amount, cost_of_loan = calculate_contract(age, maturity, interest_rate, loan_amount)

        contract = {
            'age': age,
            'loan_amount': loan_amount,
            'currency': currency.value, # Fix within one simulation
            'interest_rate': interest_rate,
            'maturity': maturity,
            'mortality_table': mortality_table.value, # Fix within one simulation
            'single_premium': single_premium,
            'annual_premium': annual_premium
        }
        random_contracts.append(contract)

    print(random_contracts)

    df = pd.DataFrame(random_contracts)
    df = df.drop(columns=['currency'])

    le = LabelEncoder()
    df['mortality_table'] = le.fit_transform(df['mortality_table'])

    X = df[['age', 'loan_amount', 'interest_rate', 'maturity', 'mortality_table']]
    y1 = df['single_premium']
    y2 = df['annual_premium']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y1_train, y1_test = train_test_split(X_scaled, y1, test_size=0.2, random_state=42)
    X_train, X_test, y2_train, y2_test = train_test_split(X_scaled, y2, test_size=0.2, random_state=42)

    model1 = RandomForestRegressor(n_estimators=100, random_state=42)
    model2 = RandomForestRegressor(n_estimators=100, random_state=42)

    model1.fit(X_train, y1_train)
    model2.fit(X_train, y2_train)

    y1_pred = model1.predict(X_test)
    y2_pred = model2.predict(X_test)

    evaluate_model(y1_test, y1_pred, "Single Premium")
    evaluate_model(y2_test, y2_pred, "Annual Premium")

    plot_learning_curve()
    mr.Stop()
else:
    # Input validation
    if age and age.value + maturity.value > 100:
        mr.Markdown(text=f'#<font color="red">ERROR: With these values no house loan is available. Please check your inputs.</font>')
        mr.Stop()

In [41]:
# Stop ploting if it is simulation
if admin_toggle.value == True:
    mr.Stop()

single_premium, annual_premium, annuity_factor_calc, monthly_premium, monthly_annual_premium, monthly_annuity_factor, number_of_payments, periodic_rate, monthly_reimbursed_amount, remaining_amount, interest, amortization, total_reimbursement_amount, cost_of_loan = calculate_contract(age.value, maturity.value, interest_rate.value, amount.value)

colors = ["#F8F9FA", "green", "#FCE700", "#00F5FF"]
mr.Markdown(text=f'#<font color="black">Yearly Payment</font>')
mr.NumberBox([
    mr.NumberBox(data=float(f"{single_premium:.2f}"), title=f"Single premium ({currency.value})", background_color=colors[0], border_color=colors[1], data_color=colors[1], title_color=colors[1]),
    mr.NumberBox(data=float(f"{annual_premium:.2f}"), title=f"Annual premium ({currency.value})", background_color=colors[0], border_color=colors[1], data_color=colors[1], title_color=colors[1]),
    mr.NumberBox(data=float(f"{annuity_factor_calc:.3f}"), title="Annuity factor")
])

#<font color="black">Yearly Payment</font>

In [42]:
mr.Markdown(text=f'#<font color="black">Monthly Payment</font>')
mr.NumberBox([
    mr.NumberBox(data=float(f"{monthly_premium:.2f}"), title=f"Monthly premium ({currency.value})"),
    mr.NumberBox(data=float(f"{monthly_annual_premium:.2f}"), title=f"Annual premium ({currency.value})"),
    mr.NumberBox(data=float(f"{monthly_annuity_factor:.3f}"), title="Annuity factor")
])

#<font color="black">Monthly Payment</font>

In [43]:
# Balance sheet for year 1
# Sum of Assets and Liability should be equal

# Assets
balance_sheet_annual_premium = annual_premium
balance_sheet_interest = balance_sheet_annual_premium * interest_rate.value / 100
balance_sheet_sum_of_assets = balance_sheet_annual_premium + balance_sheet_interest

# Liability
balance_sheet_claims = remaining_amount[12] * (1 - mortality_data[age.value + 1 ] / mortality_data[age.value])
balance_sheet_recurrence = (0 + balance_sheet_annual_premium - remaining_amount[12] * (1 / (1 + interest_rate.value / 100)) * n_1qx(age.value+12/12-1, 0)) / ((1 / (1 + interest_rate.value / 100)) * (1 -  n_1qx(age.value+12/12-1, 0)))
balance_sheet_reserves = balance_sheet_recurrence * mortality_data[age.value + 1 ] / mortality_data[age.value]
balance_sheet_sum_of_liabilities = balance_sheet_claims + balance_sheet_reserves

# Validation check: assets should be equal to liabilities
if balance_sheet_sum_of_assets - balance_sheet_sum_of_liabilities > tolerance:
    mr.Markdown(text=f'#<font color="red">ERROR: The difference between the assets and the liabilities are greater than the tolerance.</font>')

mr.Markdown(text=f'#<font color="black">Balance Sheet for Year 1</font>')

df = pd.DataFrame(
    {
        "Assets": ["Annual Premium*", "Interest (financial income)", "Opening", "Sum of Assets"],
        "Assets Value": [
            f"{float(balance_sheet_annual_premium):.2f} {currency.value}",
            f"{float(balance_sheet_interest):.2f} {currency.value}",
            f"{float(0):.2f} {currency.value}",
            f"{float(balance_sheet_sum_of_assets):.2f} {currency.value}"
        ],
        "Liability": ["Claims", "Premium Reserves", "", "Sum of Liabilities"],
        "Liability Value": [
            f"{float(balance_sheet_claims):.2f} {currency.value}",
            f"{float(balance_sheet_reserves):.2f} {currency.value}",
            "",
            f"{float(balance_sheet_sum_of_liabilities):.2f} {currency.value}"
        ]
    }
)

mr.Table(data=df, width="150px", text_align="left")

#<font color="black">Balance Sheet for Year 1</font>

In [44]:
print("* We are using annual premium with yearly payment for balance sheet calculations.")

* We are using annual premium with yearly payment for balance sheet calculations.


In [ ]:
if details_toggle.value == False:
    mr.Stop()

# Detailed calculation
mr.Markdown(text=f'#<font color="black">Detailed calculation</font>')

mr.Markdown(text=f'###<font color="black">Number of payments / Durations (month): {number_of_payments:.0f}</font>')
mr.Markdown(text=f'###<font color="black">Periodic rate: {periodic_rate * 100:.4f} %</font>')
mr.Markdown(text=f'###<font color="black">Cost of loan: {cost_of_loan:.2f} {currency.value}</font>')
mr.Markdown(text=f'###<font color="black">Total reimbursement amount: {total_reimbursement_amount:.2f} {currency.value}</font>')
mr.Markdown(text=f'#') # spacer
mr.Markdown(text=f'#') # spacer

num_rows = len(amortization)

remaining_amount = remaining_amount
interest[0] = pd.NA
amortization[0] = pd.NA
monthly_reimbursed_amount_array = [monthly_reimbursed_amount] * (num_rows)
monthly_reimbursed_amount_with_insurance_array = [monthly_reimbursed_amount + monthly_premium] * (num_rows)
beginning_remaining_amount = [pd.NA] + remaining_amount[:-1]

detailed_calc = pd.DataFrame({
    "Remaining amount (beginning of the year) (Ck-1)": beginning_remaining_amount,
    "Interest (Ik)": interest,
    "Amortization (Ak)": amortization,
    "Monthly reimbursed amount (Ek)": monthly_reimbursed_amount_array,
    "Monthly reimbursed amount with insurance": monthly_reimbursed_amount_with_insurance_array,
    "Remaining amount (end of the year) (Ck)": remaining_amount,
})

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
pd.set_option("display.expand_frame_repr", False)

detailed_calc.index.name = "Month"
styled_table = detailed_calc.style.set_table_attributes('style="display:inline"') \
                                   .set_caption("Detailed Calculation Overview") \
                                   .set_table_styles([
                                       {'selector': 'thead th', 'props': [('background-color', '#D3D3D3'), ('color', 'black'), ('font-weight', 'bold')]},
                                       {'selector': 'index', 'props': [('text-align', 'center'), ('font-weight', 'bold')]},
                                       {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('border', '1px solid black')]},
                                       {'selector': 'thead', 'props': [('border', '1px solid black')]},
                                       {'selector': 'tbody', 'props': [('border', '1px solid black')]},
                                   ])


# Display the styled DataFrame
display(styled_table)

In [ ]:
# Mortality table details
mr.Markdown(text=f'#<font color="black">Mortality table details</font>')
mr.Markdown(text=f'###<font color="black">Chosen table: {mortality_table.value}</font>')

df_mortality_data = pd.DataFrame(list(mortality_data.items()), columns=["Age (x)", "lx"])

# Display the table
df_mortality_data